In [40]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os

In [41]:
# ChromeDriver 자동 설치 및 실행
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 백그라운드 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Project Gutenberg 웹사이트 열기
driver.get("https://www.gutenberg.org/")
print("ChromeDriver 실행 성공!")

# 웹 요소가 상호 작용 가능한 상태가 될 때까지 최대 20초 동안 기다림
wait = WebDriverWait(driver, 20)

ChromeDriver 실행 성공!


In [42]:
# 검색창 찾기 (id="menu-book-search")
search_box = wait.until(EC.element_to_be_clickable((By.ID, "menu-book-search")))

# 사용자가 검색어 입력
search_term = input("검색할 단어를 입력하세요: ")

# 사용자가 입력한 검색어를 입력 후 검색 실행
search_box.send_keys(search_term)
search_box.send_keys(Keys.RETURN)
print(f"검색 완료: '{search_term}'")

# 검색 결과 페이지가 로드될 시간을 기다림
time.sleep(5)

검색 완료: 'fiction'


In [43]:
# "Bookshelves" 버튼 찾기 및 클릭
bookshelves_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Bookshelves']")))
bookshelves_link.click()

In [44]:
# "Browsing: Fiction" 링크 찾기 및 클릭
browsing_fiction_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='title' and text()='Browsing: Fiction']")))
browsing_fiction_link.click()

# 페이지가 로드될 시간을 기다림
time.sleep(5)

In [45]:
# 파일명에서 사용할 수 없는 문자 제거
def sanitize_filename(title):
    return re.sub(r'[\\/*?:"<>|\n]', '_', title).strip()

In [46]:
# 책 제목 리스트를 다시 가져오는 함수 (stale element reference 방지)
def reload_title_links():
    time.sleep(2)  # 페이지 로드 대기
    return [link for link in wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='content']/div[2]/div/ul/li/a")))
            if 'Sort' not in link.text]

In [47]:
def process_books():
    """ 현재 페이지의 책을 크롤링 """
    title_links = reload_title_links()

    # 저장할 폴더 경로 설정
    save_folder = "books"  # 원하는 폴더 이름으로 변경 가능
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)  # 폴더가 없다면 생성

    for index, link in enumerate(title_links):
        try:
            book_title = sanitize_filename(link.text)  # 파일명 정리
            print(f"{index+1}. {book_title} - 이동 중...")
            link.click()
            time.sleep(3)  # 페이지 로드 대기

            # 현재 페이지 URL 출력
            print(f"현재 페이지 URL: {driver.current_url}")

            # 'Plain Text UTF-8' 링크 클릭
            plain_text_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Plain Text UTF-8")))
            plain_text_link.click()
            time.sleep(3)  # 페이지 로드 대기

            # 본문 내용 크롤링 (최대 30,000자까지)
            book_content_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//body//pre")))

            # 크롤링된 텍스트를 하나로 합침
            full_text = "\n".join([para.text for para in book_content_elements])

            # 'CONTENTS', 'Contents', 'contents'가 본문에 포함된 경우만 크롤링 진행
            start_index = None
            for idx, text in enumerate(full_text.split("\n")):
                if any(keyword in text.lower() for keyword in ['contents']):
                    start_index = idx
                    break

            if start_index is not None:
                # 'contents'가 발견된 지점부터 크롤링
                book_text = "\n".join(full_text.split("\n")[start_index:])[:30000]

                # 마지막 줄이 마침표로 끝나는지 확인하고, 끝나지 않으면 제거
                book_text_lines = book_text.split("\n")  # 줄 단위로 나누기
                if book_text_lines:
                    last_line = book_text_lines[-1].strip()  # 마지막 줄에서 공백 제거

                    # 마지막 줄이 마침표로 끝나는지 확인
                    while not last_line.endswith(".") and len(book_text_lines) > 1:
                        book_text_lines = book_text_lines[:-1]  # 마지막 줄을 제거
                        last_line = book_text_lines[-1].strip()  # 새로운 마지막 줄을 확인

                    # 줄이 마침표로 끝날 때까지 크롤링을 이어가도록 수정
                    if len(book_text_lines) > 1:
                        book_text_lines = book_text_lines[:-1]  # 마지막 줄까지 제거하지 않고 이어서 크롤링

                # 텍스트를 다시 합침
                book_text = "\n".join(book_text_lines)

                # 파일 저장 경로
                file_path = os.path.join(save_folder, f"{book_title}.txt")
                with open(file_path, "w", encoding="utf-8") as file:
                    file.write(book_text)

                print(f"책 내용 저장 완료: {file_path} 생성 (총 {len(book_text)}자)")
            else:
                print(f"책 내용에 'contents'가 포함되지 않아 크롤링을 건너뜁니다.")

            # 뒤로 가기 및 새로고침 (stale element reference 방지)
            driver.back()
            time.sleep(3)
            driver.back()
            time.sleep(3)

        except Exception as e:
            print(f"오류 발생 (책 제목: {book_title}): {e}")
            continue

In [48]:
def go_to_next_page():
    """ 가장 마지막 'Next' 버튼을 찾아 클릭하여 다음 페이지로 이동 """
    try:
        next_buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@title='Go to the next page of results.']")))
        last_next_button = next_buttons[-1]  # 마지막 'Next' 버튼 선택
        last_next_button.click()
        time.sleep(5)  # 페이지 로드 대기
        print("📌 다음 페이지로 이동 완료.")
        return True  # 다음 페이지로 이동 성공
    except Exception:
        print("📌 다음 페이지 없음. 크롤링 종료.")
        return False  # 다음 페이지가 없으면 종료

In [49]:
# 전체 페이지 크롤링 실행
try:
    while True:
        process_books()  # 현재 페이지의 책 크롤링
        if not go_to_next_page():
            break  # 다음 페이지가 없으면 루프 종료

except Exception as e:
    print(f"전체 오류 발생: {e}")

finally:
    driver.quit()

1. Frankenstein; Or, The Modern Prometheus_Mary Wollstonecraft Shelley_86540 downloads - 이동 중...
현재 페이지 URL: https://www.gutenberg.org/ebooks/84
책 내용 저장 완료: books\Frankenstein; Or, The Modern Prometheus_Mary Wollstonecraft Shelley_86540 downloads.txt 생성 (총 28459자)
2. A Christmas Carol in Prose; Being a Ghost Story of Christmas_Charles Dickens_80910 downloads - 이동 중...
현재 페이지 URL: https://www.gutenberg.org/ebooks/46
책 내용 저장 완료: books\A Christmas Carol in Prose; Being a Ghost Story of Christmas_Charles Dickens_80910 downloads.txt 생성 (총 29567자)
3. Moby Dick; Or, The Whale_Herman Melville_71579 downloads - 이동 중...
오류 발생 (책 제목: Moby Dick; Or, The Whale_Herman Melville_71579 downloads): Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x007EEC13+23731]
	(No symbol) [0x0077C394]
	(No symbol) [0x0065BE63]
	(No symbol) [0x0063D92B]
	(No symbol) [0x006C7F7F]
	(No symbol) [0x006DAD9